In [1]:
# Step 0. Load libraries and custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import re
from dateutil.parser import parse
from datetime import datetime
from tqdm import tqdm
# ------------  PREPROCESING -------------
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#-------------- TRANSFORMERS --------------
import transformers
from transformers.pipelines import PIPELINE_REGISTRY
from transformers import pipeline
import evaluate
from evaluate import load
from transformers import Conversation
transformers.logging.set_verbosity_error()

/home/arnaldochm/Documents/BootCamp_DataScience/Final_Project/final_project_nlp/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-04 18:40:31.148707: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-04 18:40:31.157522: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-04 18:40:31.227373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-04 18:40:31.228384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimize

In [2]:
df_reduced = pd.read_csv('../data/processed/df_reduced.csv')
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502041 entries, 0 to 502040
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  502041 non-null  int64 
 1   num_row     502041 non-null  int64 
 2   text        502041 non-null  object
dtypes: int64(2), object(1)
memory usage: 11.5+ MB


In [3]:
df_reduced = df_reduced.drop(['Unnamed: 0'], axis= 1)

In [4]:
df_reduced.sample(10)

,num_row,text
245786,245786,WHAT'S NOT TO LOVE?My love of this Bible began...
173234,173234,I had this book for a Spanish literature cours...
300431,300431,55000 miles of trying to heal. What is one to ...
455899,455899,The book presents a unique opportunity for rea...
224031,224031,I concur with Mojo's review and thanks for the...
351667,351667,Ever wonder how automobiles get that perfect l...
252926,252926,This book is great for many first time parents...
353309,353309,"Have to agree some of the others reviews, this..."
13853,13853,Chloe Rand seems to have the perfect life. She...
409548,409548,Excellent book for JAR-66 Module 7 and A&amp;P...


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arnaldochm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def clean_stopwords(text: str,stop_dict: dict)->str:
    if text is not None:
        words = text.split()
        words_clean = []
        for word in words:
            if word not in stop_dict:
                words_clean.append(word)
        result = ' '.join(words_clean)
    else:
        result = None
    return result

In [7]:
# 3.10 Process text to extract stopwords
df_reduced['text_clean'] = df_reduced['text'].str.lower()
stop_dict = stopwords.words('english')
df_reduced['text_clean'] = df_reduced['text_clean'].apply(lambda x: clean_stopwords(x, stop_dict = stop_dict))

In [8]:
# 3.12 Extract special characters
df_reduced['text_clean'] = df_reduced['text_clean'].str.replace(r'''[!.,():\-%$/'"‘]''', '', regex=True)

In [9]:
# 3.13 Extract numbers
df_reduced['text_clean'] = df_reduced['text_clean'].str.replace(r'[\d]+', '', regex=True)

In [10]:
df_reduced = df_reduced.drop(['text'],axis=1)

In [11]:
# 3.14 See the results
df_reduced.head(10)

,num_row,text_clean
0,0,pretty good book clear content say something r...
1,1,terry pratchetts first novel the carpet people...
2,2,must around capable artist basil hallward bri...
3,3,first read book early teens reread fourth time...
4,4,patrick kelsey learns woman replace retired co...
5,5,someone know receives diagnosis prostate cance...
6,6,simple point chalk full information herbs uses...
7,7,read book day & half interesting reading belie...
8,8,bought book biological anthropology class text...
9,9,slaughterhouse slaughterhouse five childrens ...


In [12]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/arnaldochm/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
vaderSentimentAnalyzer = SentimentIntensityAnalyzer()

In [14]:
vaderSentimentAnalyzer.polarity_scores(df_reduced.iloc[67]['text_clean'])

{'neg': 0.162, 'neu': 0.568, 'pos': 0.27, 'compound': 0.7579}

In [15]:
df_reduced['scores']=df_reduced['text_clean'].apply(lambda body: vaderSentimentAnalyzer.polarity_scores(str(body)))
df_reduced.head()

,num_row,text_clean,scores
0,0,pretty good book clear content say something r...,"{'neg': 0.076, 'neu': 0.45, 'pos': 0.474, 'com..."
1,1,terry pratchetts first novel the carpet people...,"{'neg': 0.071, 'neu': 0.776, 'pos': 0.153, 'co..."
2,2,must around capable artist basil hallward bri...,"{'neg': 0.119, 'neu': 0.642, 'pos': 0.24, 'com..."
3,3,first read book early teens reread fourth time...,"{'neg': 0.146, 'neu': 0.699, 'pos': 0.154, 'co..."
4,4,patrick kelsey learns woman replace retired co...,"{'neg': 0.158, 'neu': 0.621, 'pos': 0.221, 'co..."


In [16]:
df_reduced['compound_sentiment']=df_reduced['scores'].apply(lambda score_dict:score_dict['compound'])
df_reduced.head()

,num_row,text_clean,scores,compound_sentiment
0,0,pretty good book clear content say something r...,"{'neg': 0.076, 'neu': 0.45, 'pos': 0.474, 'com...",0.9948
1,1,terry pratchetts first novel the carpet people...,"{'neg': 0.071, 'neu': 0.776, 'pos': 0.153, 'co...",0.9823
2,2,must around capable artist basil hallward bri...,"{'neg': 0.119, 'neu': 0.642, 'pos': 0.24, 'com...",0.9970
3,3,first read book early teens reread fourth time...,"{'neg': 0.146, 'neu': 0.699, 'pos': 0.154, 'co...",0.1280
4,4,patrick kelsey learns woman replace retired co...,"{'neg': 0.158, 'neu': 0.621, 'pos': 0.221, 'co...",0.4624


In [17]:
df_reduced = df_reduced.drop(['scores'], axis=1)
df_reduced.sample(10)

,num_row,text_clean,compound_sentiment
455827,455827,book one carry long time offers many opportuni...,0.9834
289810,289810,husband loves book recipes fry anything think ...,0.8225
226952,226952,somewhat top author sticks pins lot hypocritic...,0.2748
4832,4832,one best books read want see happens countries...,0.9360
457118,457118,another wonderful leigh koslow mystery easily ...,0.8020
379972,379972,demian hesse relates spiritual struggle one em...,0.9956
314349,314349,thoroughly enjoyed version three little pigs w...,0.9302
282052,282052,ohim sorry rate book ? probably mentten probab...,0.9495
263028,263028,gospel luke acts apostles jointly called lukea...,0.9838
404561,404561,is course admirable roth takes great effort wr...,0.9907


In [18]:
df_reduced.to_csv('../data/processed/df_reduced_with_sentiment.csv')